In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard

import torch
import accelerate
import peft
import bitsandbytes
import transformers
import trl
import tensorboard

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import pipeline


/mnt/c/Users/FAST/O_2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

base_model_name = "NousResearch/Llama-2-7b-chat-hf"
adapter_name = "Llama-2-7b-chat-finetune"

# Load base model with proper settings
model = AutoModelForCausalLM.from_pretrained(
    base_model_name, 
    torch_dtype=torch.float16,  # Use FP16 to save memory
    device_map="auto", 
)

# Load fine-tuned LoRA adapter
model = PeftModel.from_pretrained(model, adapter_name)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

/mnt/c/Users/FAST/O_2/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.55s/it]
/mnt/c/Users/FAST/O_2/lib/python3.8/site-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary obj

In [5]:
print("Loaded adapters:", model.peft_config)

Loaded adapters: {'default': LoraConfig(peft_type='LORA', auto_mapping=None, base_model_name_or_path='NousResearch/Llama-2-7b-chat-hf', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=64, target_modules=['q_proj', 'v_proj'], lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)}


In [ ]:
# Run text generation pipeline with our next model
import logging
from transformers import pipeline, logging as transformers_logging
transformers_logging.set_verbosity(transformers_logging.CRITICAL)
prompt = """I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
How can I change my feeling of being worthless to everyone?"""
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

# Test Data Evaluation (Custom dataset)

# 1st Query

In [79]:
from transformers import pipeline

# Without Chain of Thought (basic prompt)
validation_prompts_without_cot = [
    """I've been feeling so anxious lately I can barely function. My doctor prescribed medication but I'm scared to take it because of the side effects I've read about online. What should I do?"""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Model Output: Anxiety is a tough one to deal with, and it’s always a good idea to talk with a professional about it. There are some things you can try that don’t involve medication. For example, some people find it helpful to try and meditate on their anxiety, focusing on it and breathing through it. You can also try some yoga or other exercises that help you to relax. If that’s not your thing, you can also try some other relaxation strategies. There are some that involve listening to music or using a CD. One that I like is called “Relaxation Without Drugs”. If you’re interested in trying something like that, you can find it on Amazon.One thing that can be difficult with anxiety is that it can make it difficult to fall asleep. If that’s something that you are struggling with, you might want to look into some strategies for sleeping better. There are some things you can try before bed that can help you to relax and make it easier to fall asleep. One thing you can try is to use a lavende

In [80]:
reference_responses = [
    """The hesitation you're feeling about starting medication is extremely common and understandable. When anxiety is already making you feel out of control, the idea of taking something that might cause side effects can feel particularly threatening. A few perspectives that might help: 1) Online information about medication side effects often skews negative because people with neutral or positive experiences rarely write reviews. Studies show most people experience either no significant side effects or temporary mild ones that subside within days or weeks. 2) Anxiety itself can make us hypervigilant about bodily sensations, sometimes causing us to attribute normal variations to medication effects. 3) Most anxiety medications can be started at very low doses and gradually increased, which significantly reduces side effect risk. If you decide to try the medication, consider keeping a simple daily log of how you're feeling to objectively track any changes. Remember that medication is a tool - not necessarily a lifetime commitment - that can provide enough symptom relief to engage with other helpful approaches like therapy. That said, medication is always your choice. If you're not ready, discuss alternatives with your doctor such as therapy (particularly cognitive-behavioral therapy), which has excellent evidence for anxiety treatment. Whatever you decide, severe anxiety that impairs functioning typically responds well to treatment, and you don't have to continue feeling this way."""
]
from bert_score import score

# Calculate BERTScore for responses without CoT
P_without, R_without, F1_without = score(generated_responses_without_cot, reference_responses, lang='en')

# Display results for both methods
print("Without Chain of Thought:")
for i, (p, r, f1) in enumerate(zip(P_without, R_without, F1_without)):
    print(f"  Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}")


# Calculate and display average scores
print("\nAverage Scores:")
print(f"Without CoT - Precision: {P_without.mean():.4f}, Recall: {R_without.mean():.4f}, F1: {F1_without.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Without Chain of Thought:
  Precision: 0.8343, Recall: 0.8256, F1: 0.8300

Average Scores:
Without CoT - Precision: 0.8343, Recall: 0.8256, F1: 0.8300


# With Chain of thought

In [ ]:
print(model)

In [96]:
# With Chain of Thought
validation_prompts_with_cot = [
    """
Query: "I've been feeling so anxious lately I can barely function. My doctor prescribed medication but I'm scared to take it because of the side effects I've read about online. What should I do?"

Provide your response step-by-step:
1. Empathize with the user's feelings and acknowledge their courage in seeking help.
2. Identify possible sources of their feelings of worthlessness.
3. Offer actionable steps to improve their self-image and emotional well-being.
4. Conclude with encouraging words."""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Model Output: I understand that you are afraid of the side effects of the medication that was prescribed by your doctor. There are some medications that have more side effects than others. You should talk with your doctor about your concerns. There may be another medication that will work better for you with less side effects. Another option is to try therapy. There are many different types of therapy that you can choose from. One type of therapy is Cognitive Behavioral Therapy (CBT). CBT is a short term therapy that focuses on your thoughts, feelings, and behaviors. It is a good therapy to try if you are concerned about the side effects of medication. Another type of therapy that you may want to try is Interpersonal Therapy (IPT). IPT focuses on your relationships. It is a short term therapy that last about 17 sessions. Another option is Psychodynamic Therapy. It is a long term therapy that focuses on your past experiences. There are many other types of therapy that you can choose fro

In [98]:
reference_responses = [
    """The hesitation you're feeling about starting medication is extremely common and understandable. When anxiety is already making you feel out of control, the idea of taking something that might cause side effects can feel particularly threatening. A few perspectives that might help: 1) Online information about medication side effects often skews negative because people with neutral or positive experiences rarely write reviews. Studies show most people experience either no significant side effects or temporary mild ones that subside within days or weeks. 2) Anxiety itself can make us hypervigilant about bodily sensations, sometimes causing us to attribute normal variations to medication effects. 3) Most anxiety medications can be started at very low doses and gradually increased, which significantly reduces side effect risk. If you decide to try the medication, consider keeping a simple daily log of how you're feeling to objectively track any changes. Remember that medication is a tool - not necessarily a lifetime commitment - that can provide enough symptom relief to engage with other helpful approaches like therapy. That said, medication is always your choice. If you're not ready, discuss alternatives with your doctor such as therapy (particularly cognitive-behavioral therapy), which has excellent evidence for anxiety treatment. Whatever you decide, severe anxiety that impairs functioning typically responds well to treatment, and you don't have to continue feeling this way."""
]
from bert_score import score

# Calculate BERTScore for responses with CoT
P_without, R_without, F1_without = score(generated_responses_without_cot, reference_responses, lang='en')


# Display results for both methods
print("With Chain of Thought:")
for i, (p, r, f1) in enumerate(zip(P_without, R_without, F1_without)):
    print(f"Example {i+1} - Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}")


# Calculate and display average scores
print("\nAverage Scores:")
print(f" CoT - Precision: {P_without.mean():.4f}, Recall: {R_without.mean():.4f}, F1: {F1_without.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


With Chain of Thought:
Example 1 - Precision: 0.8498, Recall: 0.8280, F1: 0.8388

Average Scores:
 CoT - Precision: 0.8498, Recall: 0.8280, F1: 0.8388


# 2nd query

In [5]:
from transformers import pipeline

# Without Chain of Thought (basic prompt)
validation_prompts_without_cot = [
    """Every night I lie awake for hours thinking about all my mistakes and problems. I've tried sleeping pills but they make me groggy the next day. What can I do to quiet my mind?"""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Model Output: Sleep is a very important part of living a healthy life. I would suggest that you talk to your primary care physician about your sleeping problems. There may be other pills that will help you sleep that won't make you groggy the next day. It's important to be able to function the next day. Sleep is important for our emotional and mental health. It gives us the opportunity to process through our problems and problems in a healthy way. Sometimes sleeping problems can be related to anxiety or depression. If this is the case, I would suggest that you talk to your primary care physician about your sleeping and emotional problems. It may be that you will need additional support. There are some things that you can do to help yourself sleep better. It's important to create a sleep ritual that works for you. Some people like to read before bed, some people like to listen to music and some people like to drink herbal tea. Whatever it is that you like to do before bed, try to make i

In [7]:
reference_responses = [
    """What you're describing sounds like rumination, which is when our minds get caught in repetitive thought cycles, often focused on problems or negative experiences. This is extremely common, especially when we're facing stress or anxiety. Some strategies that may help include: 1) Creating a "worry time" earlier in the day where you deliberately think about concerns for 15-20 minutes, then practice postponing nighttime thoughts to this designated time. 2) Developing a calming pre-sleep routine that signals to your body it's time to rest - perhaps reading, gentle stretching, or meditation. 3) When thoughts arise, acknowledge them without judgment, then gently redirect your focus to your breathing or use a mental visualization like imagining thoughts as leaves floating down a stream. If these self-help approaches don't bring relief within a few weeks, consider speaking with a therapist who specializes in cognitive-behavioral therapy for insomnia (CBT-I), which has shown excellent results for this exact problem without the side effects of medication."""
]
from bert_score import score

# Calculate BERTScore for responses without CoT
P_without, R_without, F1_without = score(generated_responses_without_cot, reference_responses, lang='en')

# Display results for both methods
print("Without Chain of Thought:")
for i, (p, r, f1) in enumerate(zip(P_without, R_without, F1_without)):
    print(f"  Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}")


# Calculate and display average scores
print("\nAverage Scores:")
print(f"Without CoT - Precision: {P_without.mean():.4f}, Recall: {R_without.mean():.4f}, F1: {F1_without.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Without Chain of Thought:
  Precision: 0.8400, Recall: 0.8299, F1: 0.8349

Average Scores:
Without CoT - Precision: 0.8400, Recall: 0.8299, F1: 0.8349


# With Chain of thought

In [9]:
# With Chain of Thought
validation_prompts_with_cot = [
    """
Query: "Every night I lie awake for hours thinking about all my mistakes and problems. I've tried sleeping pills but they make me groggy the next day. What can I do to quiet my mind?"

Provide your response step-by-step:
1. Empathize with the user's feelings and acknowledge their courage in seeking help.
2. Identify possible sources of their feelings of worthlessness.
3. Offer actionable steps to improve their self-image and emotional well-being.
4. Conclude with encouraging words."""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Model Output: You're not alone. Many people struggle with insomnia, and it sounds like you may be feeling overwhelmed by the worry of "what if." It's hard to sleep when your mind is racing. Here are a few suggestions that may help:1. Journaling: Writing down your thoughts before bed may help you release the worries and allow you to sleep more peacefully.2. Meditation: There are many guided meditation apps that can help you focus your mind and quiet your thoughts before sleep.3. Yoga: Yoga combines meditation and physical movement. Some yoga practices are specifically designed for sleep.4. Aromatherapy: Lavender and Chamomile are both great oils for sleep. You can use them in an essential oil diffuser, or apply them to your skin before bed.5. Music: Listening to soothing music before bed may help you relax.6. Write down your worries: On a piece of paper, write down all your worries and concerns. Then, write a date next to each one, and mark them off as you solve them.7. Create a routine

In [10]:
reference_responses = [
    """What you're describing sounds like rumination, which is when our minds get caught in repetitive thought cycles, often focused on problems or negative experiences. This is extremely common, especially when we're facing stress or anxiety. Some strategies that may help include: 1) Creating a "worry time" earlier in the day where you deliberately think about concerns for 15-20 minutes, then practice postponing nighttime thoughts to this designated time. 2) Developing a calming pre-sleep routine that signals to your body it's time to rest - perhaps reading, gentle stretching, or meditation. 3) When thoughts arise, acknowledge them without judgment, then gently redirect your focus to your breathing or use a mental visualization like imagining thoughts as leaves floating down a stream. If these self-help approaches don't bring relief within a few weeks, consider speaking with a therapist who specializes in cognitive-behavioral therapy for insomnia (CBT-I), which has shown excellent results for this exact problem without the side effects of medication."""
]
from bert_score import score

# Calculate BERTScore for responses with CoT
P_without, R_without, F1_without = score(generated_responses_without_cot, reference_responses, lang='en')


# Display results for both methods
print("With Chain of Thought:")
for i, (p, r, f1) in enumerate(zip(P_without, R_without, F1_without)):
    print(f"Example {i+1} - Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}")


# Calculate and display average scores
print("\nAverage Scores:")
print(f" CoT - Precision: {P_without.mean():.4f}, Recall: {R_without.mean():.4f}, F1: {F1_without.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


With Chain of Thought:
Example 1 - Precision: 0.8574, Recall: 0.8426, F1: 0.8499

Average Scores:
 CoT - Precision: 0.8574, Recall: 0.8426, F1: 0.8499


# 3rd Query

In [146]:
from transformers import pipeline

# Without Chain of Thought (basic prompt)
validation_prompts_without_cot = [
    """I struggle with depression as well as pretty intense mood swings throughout the month. I experience highs where I feel amazing and energetic and then lows where I lack focus, energy, and generally have a more dark outlook on my life. How can I live a more balanced life?"""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Model Output: I think if you're feeling depressed, it would be a good idea to talk to a professional.  Depression is a serious condition that can be treated.  If left untreated it can worsen over time and affect your ability to function.  If you are having mood swings, they could be related to your depression or something else entirely.  If you're not depressed now but have been in the past, ponder on your previous episodes of mood swings during those times.  Also, pay attention to your sleeping habits.  A lack of sleep can affect your mood and energy levels.  A balanced life starts with a balanced you.  Get some rest, eat well, and stay active.  Exercise releases endorphins which can improve your mood.  Also, try to engage in activities that you enjoy.  If you're not sure what that is, maybe talk to a professional.  They can help you discover your passion and interest.  Remember, you are not alone.  Many people struggle with depression and other mental health conditions.  Don't be afr

In [33]:
reference_responses = [
    "Depression and PMS symptoms can be so hard to navigate !  Looking at this from a holistic standpoint can help you understand why you get these mood swings and what to do to balance yourself emotionally during this time. As a woman, we experience peaks and dips during our hormonal cycles and right before menstruation, we tend to feel not ourselves and down. The bloating and extreme cravings do not help either.Making sure your schedule is light and you have things to assist you can be so helpful. Different essential oils like lavender may calm you down, or orange which can uplift your mood. Homeopathy can be effective and gentle during this time as well. Birth control is known to help reduce the extreme spikes in hormones and can reduce your emotional and physical pain. Perhaps even psychiatric medication like an SSRI can help . If you are interested in going down the medication route, be sure to ask the doctor about potential side effects and risks . Reducing your processed sugar intake may help a lot as well. Exercising, spending time outside, practicing stress management and avoiding caffeine and alcohol can all be useful as well.
]
from bert_score import score

# Calculate BERTScore for responses without CoT
P_without, R_without, F1_without = score(generated_responses_without_cot, reference_responses, lang='en')


# Display results for both methods
print("Without Chain of Thought:")
for i, (p, r, f1) in enumerate(zip(P_without, R_without, F1_without)):
    print(f"Example {i+1} - Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}")


# Calculate and display average scores
print("\nAverage Scores:")
print(f"Without CoT - Precision: {P_without.mean():.4f}, Recall: {R_without.mean():.4f}, F1: {F1_without.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Without Chain of Thought:
Example 1 - Precision: 0.8371, Recall: 0.8460, F1: 0.8415

Average Scores:
Without CoT - Precision: 0.8371, Recall: 0.8460, F1: 0.8415


# With Chain of Thought

In [139]:
# With Chain of Thought
validation_prompts_with_cot = [
    """
Query: "I struggle with depression as well as pretty intense mood swings throughout the month. I experience highs where I feel amazing and energetic and then lows where I lack focus, energy, and generally have a more dark outlook on my life. How can I live a more balanced life?"

Provide your response step-by-step:
1. Empathize with the user's feelings and acknowledge their courage in seeking help.
2. Identify possible sources of their feelings of worthlessness.
3. Offer actionable steps to improve their self-image and emotional well-being.
4. Conclude with encouraging words."""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

Model Output: It sounds like you are having some difficulty with maintaining a consistent balance in your mood. This is actually very common and can be very frustrating at the same time. There are some things you can try that may help you to improve your balance and reduce some of the intense fluctuations you are currently experiencing.I would start by keeping an accurate journal of your daily activities, the foods you eat, and your sleep patterns. Sometimes, making changes in these areas can have a positive impact on your mood. Additionally, if you are currently taking any prescription medications, it may be helpful to talk with your doctor about possibly making some small changes.Sometimes, the changes we make may not necessarily be enough to have a positive impact on our mood, in which case it may be helpful to look at some other options. One of the things that often helps is to try to focus on some of the things that you do every day that make you happy. For example, if you have a 

In [83]:
reference_responses = [
    "Depression and PMS symptoms can be so hard to navigate !  Looking at this from a holistic standpoint can help you understand why you get these mood swings and what to do to balance yourself emotionally during this time. As a woman, we experience peaks and dips during our hormonal cycles and right before menstruation, we tend to feel not ourselves and down. The bloating and extreme cravings do not help either.Making sure your schedule is light and you have things to assist you can be so helpful. Different essential oils like lavender may calm you down, or orange which can uplift your mood. Homeopathy can be effective and gentle during this time as well. Birth control is known to help reduce the extreme spikes in hormones and can reduce your emotional and physical pain. Perhaps even psychiatric medication like an SSRI can help . If you are interested in going down the medication route, be sure to ask the doctor about potential side effects and risks . Reducing your processed sugar intake may help a lot as well. Exercising, spending time outside, practicing stress management and avoiding caffeine and alcohol can all be useful as well."
]
from bert_score import score

# Calculate BERTScore for responses without CoT
P_without, R_without, F1_without = score(generated_responses_without_cot, reference_responses, lang='en')


# Display results for both methods
print("Without Chain of Thought:")
for i, (p, r, f1) in enumerate(zip(P_without, R_without, F1_without)):
    print(f"Example {i+1} - Precision: {p:.4f}, Recall: {r:.4f}, F1: {f1:.4f}")


# Calculate and display average scores
print("\nAverage Scores:")
print(f"Without CoT - Precision: {P_without.mean():.4f}, Recall: {R_without.mean():.4f}, F1: {F1_without.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Without Chain of Thought:
Example 1 - Precision: 0.8573, Recall: 0.8462, F1: 0.8517

Average Scores:
Without CoT - Precision: 0.8573, Recall: 0.8462, F1: 0.8517


# Urdu Prompt Checking

In [3]:
# Import necessary libraries
import logging
from transformers import pipeline, logging as transformers_logging

# Suppress warnings
transformers_logging.set_verbosity(transformers_logging.CRITICAL)

# Urdu prompt
prompt = """میں شدید ڈپریشن اور بے چینی کا شکار ہوں اور مجھے ایسا لگتا ہے کہ میں بہت زیادہ مشکلات سے گزر رہا ہوں۔
یہ چیزیں مجھے بہت پریشان کرتی ہیں اور میں ان خیالات سے جان نہیں چھڑا پا رہا جو مجھے تکلیف دے رہے ہیں۔
میں اس بے چینی اور ڈپریشن پر کیسے قابو پا سکتا ہوں؟"""

# Strong Urdu instruction
formatted_prompt = f"[INST] آپ کا جواب مکمل طور پر اردو میں ہونا چاہیے اور کسی بھی انگریزی لفظ یا جملے کی اجازت نہیں ہے۔ براہ کرم سوال کا اردو میں مفصل جواب دیں: {prompt} [/INST]"

# Initialize pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

# Generate response
result = pipe(formatted_prompt)

# Extract and clean the generated response
generated_text = result[0]['generated_text']
if "[/INST]" in generated_text:
    generated_text = generated_text.split("[/INST]", 1)[-1].strip()  # Remove prompt part

# Print the cleaned response
print("Model Output:", generated_text)


/mnt/c/Users/FAST/O_2/lib/python3.8/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Model Output: Hi there! I'm sorry to hear about the frustration you're feeling with your family. I know it can be difficult to navigate family dynamics. One thing that can be helpful is to be aware of your own emotions and to recognize the feelings that are causing you to feel frustrated. Are you feeling angry, hurt, confused, or some other emotion? Once you can recognize your emotion, you can try to figure out where it is coming from. For example, if you're feeling angry, are you angry because you feel like your family is not respecting your wishes or boundaries? Another thing that can be helpful is to try to communicate with your family in a way that is assertive. Assertive communication means that you are being clear about what you want or expect, but you are also respecting the needs of others. For example, you could say something like "I would really appreciate it if you would ask me what I'm thinking or feeling before making assumptions about me."This can be really hard to do, es

In [8]:
# Import necessary libraries
import logging
from transformers import pipeline, logging as transformers_logging

# Suppress warnings
transformers_logging.set_verbosity(transformers_logging.CRITICAL)

# Urdu prompt
prompt = """میں شدید ڈپریشن اور بے چینی کا شکار ہوں اور مجھے ایسا لگتا ہے کہ میں بہت زیادہ مشکلات سے گزر رہا ہوں۔
یہ چیزیں مجھے بہت پریشان کرتی ہیں اور میں ان خیالات سے جان نہیں چھڑا پا رہا جو مجھے تکلیف دے رہے ہیں۔
میں اس بے چینی اور ڈپریشن پر کیسے قابو پا سکتا ہوں؟"""

# Strong Urdu instruction
formatted_prompt = f"[INST] آپ کا جواب مکمل طور پر اردو میں ہونا چاہیے اور کسی بھی انگریزی لفظ یا جملے کی اجازت نہیں ہے۔ براہ کرم سوال کا اردو میں مفصل جواب دیں: {prompt} [/INST]"

# Initialize pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

# Generate response
result = pipe(formatted_prompt)

# Extract and clean the generated response
generated_text = result[0]['generated_text']
if "[/INST]" in generated_text:
    generated_text = generated_text.split("[/INST]", 1)[-1].strip()  # Remove prompt part

# Print the cleaned response
print("Model Output:", generated_text)


Model Output: What you are going through is normal. We all go through such feelings of inferiority at one time or another. The first step is to accept that you are doing your best. No one can do more than their best. Secondly, no one is perfect. Everyone has flaws. You are not alone. Lastly, no matter how hard you try, you will always find things to criticize about yourself. No one is immune to self-criticism. It is important to learn to accept and love yourself the way you are. Take care of yourself. Exercise, eat well, sleep well. Lastly, try to find a hobby or something that you enjoy doing. This will help you to focus on something positive. Best of luck! :) :)))))


In [4]:
from transformers import pipeline

# Without Chain of Thought (basic prompt)
validation_prompts_without_cot = [
    """میں شدید ڈپریشن اور بے چینی کا شکار ہوں اور مجھے ایسا لگتا ہے کہ میں بہت زیادہ مشکلات سے گزر رہا ہوں۔ یہ چیزیں مجھے بہت پریشان کرتی ہیں اور میں ان خیالات سے جان نہیں چھڑا پا رہا جو مجھے تکلیف دے رہے ہیں۔ میں اس بے چینی اور ڈپریشن پر کیسے قابو پا سکتا ہوں؟"""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

Model Output: Hi,It's normal to feel down from time to time, and it's also normal to have a lot of self-criticism.Ћeh, you can't escape those things, and you'll always have them. The question is, do you want to learn how to live with them, or do you want them to control you? You have the power to change how much power they have over you. There are some things you can do:1. Practice self-compassion.2. Learn to recognize when your self-criticism is a bit much.3. Find activities that make you feel good, and do them regularly.4. Challenge your negative thoughts and try to be more balanced in your thinking.This is a long process, and you might want to work with a therapist who can help you with it. Best of luck! :)Resources:Here's a great TED talk on the topic:https://www.ted.com/talks/lily_singer_the_mystery_of_compassion?language=enHere's a quiz to help you challenge your negative thoughts:http://www.c


In [9]:
from transformers import pipeline

# Without Chain of Thought (basic prompt)
validation_prompts_without_cot = [
    """میں اپنے جذبات اور خود سے متعلق کچھ مسائل سے گزر رہا ہوں۔ میں بہت کم سوتا ہوں اور بس یہی سوچتا رہتا ہوں کہ میں بے وقعت ہوں اور مجھے یہاں نہیں ہونا چاہیے۔
میں نے کبھی خودکشی کی کوشش نہیں کی اور نہ ہی اس بارے میں سوچا۔ میں ہمیشہ اپنی مشکلات کو حل کرنا چاہتا تھا، لیکن میں کبھی اس پر عمل نہیں کر پایا۔
میں اپنی اس سوچ کو کیسے بدل سکتا ہوں کہ میں دوسروں کے لیے بے وقعت ہوں؟
"""
]

# Initialize the text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                max_new_tokens=250, temperature=0.7, top_p=0.9, do_sample=True)

# Generate responses
generated_responses_without_cot = []
for prompt in validation_prompts_without_cot:
    result = pipe(f"[INST] {prompt} [/INST]")
    
    # Extract only the generated response
    generated_text = result[0]['generated_text']
    
    # Remove the prompt and everything before `[/INST]`
    if "[/INST]" in generated_text:
        generated_text = generated_text.split("[/INST]", 1)[-1].strip()
    
    print("Model Output:", generated_text)  # Now only the response is printed
    generated_responses_without_cot.append(generated_text)

Model Output: I'm not sure what you are asking.  Are you asking how to change the way you think about yourself?  Or are you asking how to change your negative thoughts about yourself?  Or are you asking how to change yourself?  If you could give me a little more information I would be happy to help.  Sending best wishes. ~Mark (www.MarkChannin.com and www.ChanninCounseling.com)   ~Mark   (888) 507-7886  www.MarkChannin.com  www.ChanninCounseling.com  ~Mark  MHSA, LMHC, LPC, NCC   Yonkers, NY     New York, NY  Puerto Rico    (888) 507-7886     ~Mark  MHSA, LMHC, LPC, NCC  Yonkers, NY  New York, NY  Puerto Rico    (888) 507-7886      www.MarkChannin.com


# ***********************************************************************